# Projeto 3: Segmentação de clientes no ecommerce


Realizar uma segmentação de clientes aplicando a metodologia RFM para um e-commerce.

# 1 - Contexto
 
O dataset utilizado neste notebook pode ser obtido [nesse projeto no Kaggle](https://www.kaggle.com/datasets/datacertlaboratoria/projeto-3-segmentao-de-clientes-no-ecommerce).





Este notebook está divido em 4 partes:

### Pré processamento dos dados

- Revisar a qualidade do banco de dados, remover dados duplicados que não façam sentido, encontrar dados ausentes e substituí-los/removê-los e procurar dados não correspondentes (por exemplo, números negativos em colunas que normalmente são números positivos). 

- O pré-processamento dos dados é uma etapa fundamental e básica em todas as análises de dados.

O item 2 neste notebook abrangerá essa parte.

### Análise Exploratória

- Utilizar uma ferramenta de visualização, no caso o Google Data Studio, e fazer gráficos, resumir e estruturar as informações mais relevantes.

- A ideia é poder mostrar visualmente as principais características do conjunto de dados, com foco em métricas de vendas como as listadas por seu chefe no briefing.

O item 3 irá auxiliar na criação dos gráficos.

### Análise de Coorte

- Aplicar uma análise de coorte para clientes britânicos e estrangeiros para ver qual dos dois públicos tem os clientes mais leais.

O item 4 neste notebook irá gerar o gráfico de coorte dos clientes.

### Segmentação de clientes

- Segmentar seus clientes com base em três características: Recência, Frequência e Valor Monetário.

O item 5 irá apresentar a segmentação dos clientes e terá como resultado a categorização deles.

Como resultado final será gerado um arquivo CSV para apoio nas análises em ferramentas de visualização.

# 2 - Importar, entender o dataset e limpar os dados

## Importando as bibliotecas 

As bibliotecas utilizadas neste projeto são: [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html), [NumPy](https://numpy.org/).

In [129]:
import pandas as pd
import numpy as np


In [130]:

df = pd.read_csv("./dados/vendas-por-fatura.csv")

Analisando o dataset importado

In [131]:
df.head()

,N° da fatura,Data da fatura,ID Cliente,País,Quantidade,Valor
0,548370,3/30/2021 16:14:00,15528.0,United Kingdom,123,"229,33"
1,575767,11/11/2021 11:11:00,17348.0,United Kingdom,163,"209,73"
2,C570727,10/12/2021 11:32:00,12471.0,Germany,-1,"-1,45"
3,549106,4/6/2021 12:08:00,17045.0,United Kingdom,1,"39,95"
4,573112,10/27/2021 15:33:00,16416.0,United Kingdom,357,"344,83"


In [132]:
df.columns

Index(['N° da fatura', 'Data da fatura', 'ID Cliente', 'País', 'Quantidade',
       'Valor'],
      dtype='object')

O dataset possui as 6 colunas: 'N° da fatura', 'Data da fatura', 'ID Cliente', 'País', 'Quantidade' e 'Valor' referente as faturas criadas sobre as vendas para os clientes.

Iremos renomear as colunas para facilitar a sua manipulação.

In [133]:
# Renomeando as colunas
df.columns = df.columns.str.replace(" ", "_")
df.columns

Index(['N°_da_fatura', 'Data_da_fatura', 'ID_Cliente', 'País', 'Quantidade',
       'Valor'],
      dtype='object')

Agora iremos analisar os tipos de dados:

In [134]:
# analisando os tipos de dados 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25953 entries, 0 to 25952
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   N°_da_fatura    25953 non-null  object 
 1   Data_da_fatura  25953 non-null  object 
 2   ID_Cliente      22229 non-null  float64
 3   País            25953 non-null  object 
 4   Quantidade      25953 non-null  int64  
 5   Valor           25953 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.2+ MB


## 2.1 - Detecção de nulos ou vazios

É possível analisar que existem apenas 22229 linhas com dados sobre 'ID Cliente' enquanto há 25953 linhas preenchidas referente as outras colunas. Ou seja, temos que existem 3.724 registros que não conseguimos identificar quem é o cliente. 

Como desejamos realizar a análise de coorte, é muito importante termos certeza que **NÃO** estaremos contabilizando clientes repetidos. Por isso, iremos remover esses registros com 'ID Cliente' nulos.

In [135]:
# Removendo linhas com registro nulo na coluna 'ID_Cliente'
df = df.dropna(subset=["ID_Cliente"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22229 entries, 0 to 25951
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   N°_da_fatura    22229 non-null  object 
 1   Data_da_fatura  22229 non-null  object 
 2   ID_Cliente      22229 non-null  float64
 3   País            22229 non-null  object 
 4   Quantidade      22229 non-null  int64  
 5   Valor           22229 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.2+ MB


In [136]:
# verificando se existem valores nulos
temp = df.isnull().values.any()
temp

False

Também é possível validar que não há campos nulos em nenhuma das colunas existentes.

## 2.2 Detecção e remoção de dados duplicados

In [137]:
# Removendo os dados duplicados do dataset
df.drop_duplicates(inplace=True)

df.shape

(22221, 6)

Foram identificadas e removidas 8 linhas duplicadas. Após isso temos 22221 registros únicos no nosso dataset.

## 2.3 Detecção e eliminação de quantidades negativas

Como as faturas que iniciam com a letra 'C' correspondem a devolução e desejamos analisar as vendas do negócio, então iremos identificar e remover esses valores do nosso dataset.

In [138]:
devolucao = df['N°_da_fatura'].str.startswith('C')
df[devolucao].head()


,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor
2,C570727,10/12/2021 11:32:00,12471.0,Germany,-1,"-1,45"
10,C543628,2/10/2021 15:51:00,16558.0,United Kingdom,-5,"-9,25"
17,C553508,5/17/2021 13:44:00,14000.0,United Kingdom,-3,"-1,95"
20,C577693,11/21/2021 11:48:00,12628.0,Germany,-3,"-16,85"
22,C547309,3/22/2021 11:17:00,17416.0,United Kingdom,-12,"-45,00"


In [139]:
df[devolucao].shape

(3655, 6)

Foram identificadas 3655 compras que tiveram devolução, sendo assim elas serão removidas do dataset.

In [140]:
df.drop(df[devolucao].index, inplace=True)
df.head()

,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor
0,548370,3/30/2021 16:14:00,15528.0,United Kingdom,123,"229,33"
1,575767,11/11/2021 11:11:00,17348.0,United Kingdom,163,"209,73"
3,549106,4/6/2021 12:08:00,17045.0,United Kingdom,1,"39,95"
4,573112,10/27/2021 15:33:00,16416.0,United Kingdom,357,"344,83"
5,576630,11/16/2021 8:38:00,13816.0,Germany,91,"199,98"


In [141]:
df.shape

(18566, 6)

Como resultado temos 18566 registros após aplicar a limpeza nos dados.

## Tratando os dados

Agora iremos realizar os tratamentos necessários para realizar as avaliações e cálculos necessários.

In [142]:
# Convertendo para tipo inteiro o 'ID_Cliente'
df['ID_Cliente'] = df['ID_Cliente'].astype(int)

In [143]:
# Convertendo tipo da 'Data da Fatura' para data
df["Data_da_fatura"] = pd.to_datetime(df["Data_da_fatura"])

In [144]:
# Ordenando pela Data da Fatura de maneira ascendente
df.sort_values(by=['Data_da_fatura'], inplace=True)
df.head()


,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor
9367,536365,2020-12-01 08:26:00,17850,United Kingdom,40,"139,12"
18259,536366,2020-12-01 08:28:00,17850,United Kingdom,12,"22,20"
11185,536368,2020-12-01 08:34:00,13047,United Kingdom,15,"70,05"
6876,536367,2020-12-01 08:34:00,13047,United Kingdom,83,"278,73"
8195,536369,2020-12-01 08:35:00,13047,United Kingdom,3,"17,85"


In [145]:
# alterando a separação dos valores de ',' para '.'
df['Valor'] = df['Valor'].str.replace(",", ".")
df.head()

,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor
9367,536365,2020-12-01 08:26:00,17850,United Kingdom,40,139.12
18259,536366,2020-12-01 08:28:00,17850,United Kingdom,12,22.20
11185,536368,2020-12-01 08:34:00,13047,United Kingdom,15,70.05
6876,536367,2020-12-01 08:34:00,13047,United Kingdom,83,278.73
8195,536369,2020-12-01 08:35:00,13047,United Kingdom,3,17.85


In [146]:
# alterando o tipo da coluna 'Valor' para float
df['Valor'] = pd.to_numeric(df['Valor'])
df.head()


,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor
9367,536365,2020-12-01 08:26:00,17850,United Kingdom,40,139.12
18259,536366,2020-12-01 08:28:00,17850,United Kingdom,12,22.20
11185,536368,2020-12-01 08:34:00,13047,United Kingdom,15,70.05
6876,536367,2020-12-01 08:34:00,13047,United Kingdom,83,278.73
8195,536369,2020-12-01 08:35:00,13047,United Kingdom,3,17.85


In [147]:
# Obtendo Ano e Mês
df['Ano_mes'] = df['Data_da_fatura'].dt.to_period('M')
df.head()

,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor,Ano_mes
9367,536365,2020-12-01 08:26:00,17850,United Kingdom,40,139.12,2020-12
18259,536366,2020-12-01 08:28:00,17850,United Kingdom,12,22.20,2020-12
11185,536368,2020-12-01 08:34:00,13047,United Kingdom,15,70.05,2020-12
6876,536367,2020-12-01 08:34:00,13047,United Kingdom,83,278.73,2020-12
8195,536369,2020-12-01 08:35:00,13047,United Kingdom,3,17.85,2020-12


In [148]:
# Classificando se o cliente pertence ou não ao Reino Unido
df.loc[df['País']=="United Kingdom", 'Pertence_UK'] = "SIM"
df.loc[df['País']!="United Kingdom", 'Pertence_UK'] = "NÃO"
df.head(10)

,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor,Ano_mes,Pertence_UK
9367,536365,2020-12-01 08:26:00,17850,United Kingdom,40,139.12,2020-12,SIM
18259,536366,2020-12-01 08:28:00,17850,United Kingdom,12,22.20,2020-12,SIM
11185,536368,2020-12-01 08:34:00,13047,United Kingdom,15,70.05,2020-12,SIM
6876,536367,2020-12-01 08:34:00,13047,United Kingdom,83,278.73,2020-12,SIM
8195,536369,2020-12-01 08:35:00,13047,United Kingdom,3,17.85,2020-12,SIM
505,536370,2020-12-01 08:45:00,12583,France,449,855.86,2020-12,NÃO
12932,536371,2020-12-01 09:00:00,13748,United Kingdom,80,204.00,2020-12,SIM
2773,536372,2020-12-01 09:01:00,17850,United Kingdom,12,22.20,2020-12,SIM
20456,536373,2020-12-01 09:02:00,17850,United Kingdom,88,259.86,2020-12,SIM
24617,536374,2020-12-01 09:09:00,15100,United Kingdom,32,350.40,2020-12,SIM


In [149]:
df.dtypes

N°_da_fatura              object
Data_da_fatura    datetime64[ns]
ID_Cliente                 int32
País                      object
Quantidade                 int64
Valor                    float64
Ano_mes                period[M]
Pertence_UK               object
dtype: object

# Análise de Coorte

Aqui vai ser apresentado a análise de coorte dos clientes.

In [151]:
# contando clientes únicos por ano/mês
df_coorte = pd.DataFrame(df.groupby(['Ano_mes'])['ID_Cliente'].nunique()).reset_index()
df_coorte

,Ano_mes,ID_Cliente
0,2020-12,885
1,2021-01,741
2,2021-02,758
3,2021-03,974
4,2021-04,856
5,2021-05,1056
6,2021-06,991
7,2021-07,949
8,2021-08,935
9,2021-09,1266


# 5 - Segmentação de clientes

Aqui será realizada a categorização dos clientes.


## Recencia



In [152]:
# Criando coluna de Recencia 
df_recencia = df.groupby(by='ID_Cliente',
                        as_index=False)['Data_da_fatura'].max()
df_recencia


,ID_Cliente,Data_da_fatura
0,12346,2021-01-18 10:01:00
1,12347,2021-12-07 15:52:00
2,12348,2021-09-25 13:13:00
3,12349,2021-11-21 09:51:00
4,12350,2021-02-02 16:01:00
...,...,...
4334,18280,2021-03-07 09:52:00
4335,18281,2021-06-12 10:53:00
4336,18282,2021-12-02 11:43:00
4337,18283,2021-12-06 12:02:00


In [153]:
df_recencia = df.groupby(by='ID_Cliente',
                        as_index=False)['Data_da_fatura', 'País', 'Quantidade', 'Valor'].max()
df_recencia


C:\Users\carla\AppData\Local\Temp/ipykernel_20020/3388802863.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_recencia = df.groupby(by='ID_Cliente',


,ID_Cliente,Data_da_fatura,País,Quantidade,Valor
0,12346,2021-01-18 10:01:00,United Kingdom,74215,77183.60
1,12347,2021-12-07 15:52:00,Iceland,676,1294.32
2,12348,2021-09-25 13:13:00,Finland,1254,892.80
3,12349,2021-11-21 09:51:00,Italy,631,1757.55
4,12350,2021-02-02 16:01:00,Norway,197,334.40
...,...,...,...,...,...
4334,18280,2021-03-07 09:52:00,United Kingdom,45,180.60
4335,18281,2021-06-12 10:53:00,United Kingdom,54,80.82
4336,18282,2021-12-02 11:43:00,United Kingdom,75,100.21
4337,18283,2021-12-06 12:02:00,United Kingdom,251,313.65


In [154]:
df_recencia.columns = ['Cliente', 'Data_ultima_compra']
ultima_compra = df_recencia['Data_ultima_compra'].max()
df_recencia['Recencia'] = df_recencia['Data_ultima_compra'].apply(
    lambda x: (ultima_compra - x).days)
df_recencia.head()

ValueError: Length mismatch: Expected axis has 5 elements, new values have 2 elements

## Frequência

In [ ]:
df_frequencia = df.drop_duplicates().groupby(
    by=['ID_Cliente'], as_index=False)['Data_da_fatura'].count()
df_frequencia.columns = ['Cliente', 'Frequencia']
df_frequencia.head()

,Cliente,Frequencia
0,12346,1
1,12347,7
2,12348,4
3,12349,1
4,12350,1


## Valor Monetário


In [ ]:
df['Total'] = df['Valor'] * df['Quantidade']
df_monetario = df.groupby(by='ID_Cliente', as_index=False)['Total'].sum()
df_monetario.columns = ['Cliente', 'Valor Monetario']
df_monetario.head()

,Cliente,Valor Monetario
0,12346,5.728181e+09
1,12347,1.839237e+06
2,12348,1.422256e+06
3,12349,1.109014e+06
4,12350,6.587680e+04


In [ ]:
df_rf = df_recencia.merge(df_frequencia, on='Cliente')
df_rfm = df_rf.merge(df_monetario, on='Cliente').drop(
    columns='Data_ultima_compra')
df_rfm.head()

,Cliente,Recencia,Frequencia,Valor Monetario
0,12346,325,1,5.728181e+09
1,12347,1,7,1.839237e+06
2,12348,74,4,1.422256e+06
3,12349,18,1,1.109014e+06
4,12350,309,1,6.587680e+04


In [ ]:

df_rfm['R_rank'] = df_rfm['Recencia'].rank(ascending=False)
df_rfm['F_rank'] = df_rfm['Frequencia'].rank(ascending=True)
df_rfm['M_rank'] = df_rfm['Valor Monetario'].rank(ascending=True)
 
# normalizing the rank of the customers
df_rfm['R_rank_norm'] = (df_rfm['R_rank']/df_rfm['R_rank'].max())*100
df_rfm['F_rank_norm'] = (df_rfm['F_rank']/df_rfm['F_rank'].max())*100
df_rfm['M_rank_norm'] = (df_rfm['F_rank']/df_rfm['M_rank'].max())*100
 
df_rfm.drop(columns=['R_rank', 'F_rank', 'M_rank'], inplace=True)
 
df_rfm.head()

,Cliente,Recencia,Frequencia,Valor Monetario,R_rank_norm,F_rank_norm,M_rank_norm
0,12346,325,1,5.728181e+09,3.750291,17.204425,17.204425
1,12347,1,7,1.839237e+06,97.915211,85.492049,85.492049
2,12348,74,4,1.422256e+06,38.504542,69.785665,69.785665
3,12349,18,1,1.109014e+06,74.120662,17.204425,17.204425
4,12350,309,1,6.587680e+04,5.369206,17.204425,17.204425


In [ ]:
df_rfm['RFM_Score'] = 0.15*df_rfm['R_rank_norm']+0.28 * \
    df_rfm['F_rank_norm']+0.57*df_rfm['M_rank_norm']
df_rfm['RFM_Score'] *= 0.05
df_rfm = df_rfm.round(2)
df_rfm[['Cliente', 'RFM_Score']].head(7)

,Cliente,RFM_Score
0,12346,0.76
1,12347,4.37
2,12348,3.25
3,12349,1.29
4,12350,0.77
5,12352,4.19
6,12353,0.86


## Tabela de Classificação de Categorias

As categorias utilizadas neste trabalho foram:

- rfm score >4.5 : Top Customer
- 4.5 > rfm score > 4 : High Value Customer
- 4>rfm score >3 : Medium value customer
- 3>rfm score>1.6 : Low-value customer
- rfm score<1.6 :Lost Customer

In [ ]:

df_rfm["Categoria"] = np.where(df_rfm['RFM_Score'] >
                                      4.5, "Top Customers",
                                      (np.where(
                                        df_rfm['RFM_Score'] > 4,
                                        "High value Customer",
                                        (np.where(
    df_rfm['RFM_Score'] > 3,
                             "Medium Value Customer",
                             np.where(df_rfm['RFM_Score'] > 1.6,
                            'Low Value Customers', 'Lost Customers'))))))
df_rfm[['Cliente', 'RFM_Score', 'Categoria']].head(20)

,Cliente,RFM_Score,Categoria
0,12346,0.76,Lost Customers
1,12347,4.37,High value Customer
2,12348,3.25,Medium Value Customer
3,12349,1.29,Lost Customers
4,12350,0.77,Lost Customers
5,12352,4.19,High value Customer
6,12353,0.86,Lost Customers
7,12354,0.84,Lost Customers
8,12355,0.85,Lost Customers
9,12356,3.05,Medium Value Customer


In [156]:
# Salvando arquivo csv dos dados tratados
diretorio = "C:\\Users\\carla\\Documents\\Git\\certificacao-analista-dados-ibm-laboratoria\\projetos\\03 - Segmentação de clientes no ecommerce\\dados\\analise-vendas.csv"
df.to_csv(diretorio, sep=';', index=False, encoding='UTF-8')